In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from sklearn.linear_model import LinearRegression
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt

In [15]:
# Retreive opening day rosters
file_name = 'In_Season/Data/opening_day_rosters.pickle'
with open(file_name, 'rb') as f:
    opening_day_rosters = pickle.load(f)

# Retreive RAPTOR-VORP function
file_name = 'Model_Build/Data/vorp_raptor_regression.pickle'
with open(file_name, 'rb') as f:
    model = pickle.load(f)

# Creating functions to adjust names from VORP predictors DF
def name_adjustment(x):
    x = x.split(' ')
    x = x[0] + ' ' + x[1]      
    return x
def name_exceptions(x):
    if x == 'enes kanter':
        return 'enes freedom'
    if x == 'nicolas claxton':
        return 'nic claxton'
    if x == 'kj martin':
        return 'kenyon martin'
    if x == 'jayden scrubb':
        return 'jay scrubb'
    if x == 'mohamed bamba':
        return 'mo bamba'
    if x == 'moe wagner':
        return 'moritz wagner'
    if x == 'ogugua anunoby':
        return 'og anunoby'
    if x == 'raulzinho neto':
        return 'raul neto'
    return x

def name_standardization(df):

    # Apply various functions to standardize the names of the players
    df['Player'] = df.Player.apply(unidecode.unidecode)
    df['Player'] = df.Player.apply(name_adjustment)
    df['Player'] = df.Player.str.replace('.', '')
    df['Player'] = df.Player.str.replace("'", '')
    df['Player'] = df.Player.str.lower()
    df['Player'] = df.Player.apply(lambda x: x.split(' ')[0] + ' ' + x.split(' ')[1])

    return df

# Retreiving RAPTOR predictions  
raptor_predictions = pd.read_csv('In_Season/Data/raptor_predictions.csv')
raptor_predictions = name_standardization(raptor_predictions)

In [40]:
# Creating team VORP DF and team vorp dictionary
missed_players = list()
vorps = pd.DataFrame(columns = ['Team', 'VORP_projection'])
team_vorp_df_dict = {}
for key, value in opening_day_rosters.items():
    team = key
    team_vorp = 0
    team_vorp_df = pd.DataFrame(columns = ['Player', 'VORP'])
    for player in value:
        
        player = str(player)
        if player == 'nan':
            continue
        
        # Formatting player strings
        player = unidecode.unidecode(player)
        player = player.lower()
        player = player.replace("'", '')
        player = player.replace('.', '')
        player = player.split(' ')
        player = player[0] + ' ' + player[1]
        player = player.strip('*,')

        # Calculating predicted VORP or adding to missed players
        predictors = raptor_predictions[raptor_predictions.Player == player]
        if len(predictors) > 0:
            predictors = predictors[['RAPTOR']]
            vorp_prediction = model.predict(predictors)
            team_vorp += vorp_prediction
            series = pd.Series([player, vorp_prediction], index = team_vorp_df.columns)
            team_vorp_df = team_vorp_df.append(series, ignore_index = True)
        else:
            missed_players.append(team + ': ' + player)
            series = pd.Series([player, 0], index = team_vorp_df.columns)
            team_vorp_df = team_vorp_df.append(series, ignore_index = True)

    # Adding team vorp to overall df  
    series = pd.Series([team, team_vorp], index = vorps.columns)
    vorps = vorps.append(series, ignore_index = True)

    # Adding to team vorp dictionary
    team_vorp_df.sort_values(by = ['VORP'], ascending = False, inplace = True)
    team_vorp_df.reset_index(drop = True, inplace = True)
    team_vorp_df_dict[team] = team_vorp_df

In [41]:
missed_players

['Boston Celtics: enes freedom',
 'Brooklyn Nets: cam thomas',
 'Brooklyn Nets: dayron sharpe',
 'Brooklyn Nets: james johnson',
 'Brooklyn Nets: blake griffin',
 'Charlotte Hornets: gordon hayward',
 'Chicago Bulls: tyler cook',
 'Cleveland Cavaliers: kevin pangos',
 'Cleveland Cavaliers: cedi osman',
 'Cleveland Cavaliers: dylan windler',
 'Denver Nuggets: demarcus cousins',
 'Detroit Pistons: justin robinson',
 'Detroit Pistons: saddiq bey',
 'Detroit Pistons: trey lyles',
 'Detroit Pistons: isaiah stewart',
 'Detroit Pistons: kelly olynyk',
 'Golden State Warriors: kevon looney',
 'Golden State Warriors: nemanja bjelica',
 'Indiana Pacers: goga bitadze',
 'Los Angeles Lakers: lebron james',
 'Los Angeles Lakers: carmelo anthony',
 'Los Angeles Lakers: dwight howard',
 'Los Angeles Lakers: deandre jordan',
 'New Orleans Pelicans: herbert jones',
 'Philadelphia 76ers: furkan korkmaz',
 'Philadelphia 76ers: charlie brown',
 'Philadelphia 76ers: tobias harris',
 'Phoenix Suns: ish wain

In [ ]:
# Savings opening day VORPs DF
if save:
    file_name = 'In_Season/Data/opening_day_vorps.csv'
    vorps.to_csv(file_name)